In [1]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy

In [2]:
# def function to scrape internsg 
def internSgScrapper(pages=39, output_file_path='internSG_jobs.csv'):
    # get all pages first 
    list_of_all_pages = []
    for i in range(1,pages):
        url = f'https://www.internsg.com/jobs/{i}/?f_p=107&f_i&filter_s#isg-top'
        list_of_all_pages.append(url)

    # get title of all jobs 
    data = []
    for url in list_of_all_pages:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        divs = soup.find_all('div', class_='ast-col-lg-3')
        for div in divs:
            # Try to find an <a> tag within the <div>
            a_tag = div.find('a')
            # If an <a> tag is found and it has a 'href' attribute
            if a_tag and 'href' in a_tag.attrs:
                # Extract the URL and the job title
                url = a_tag['href']
                job_title = a_tag.get_text().strip()
                # Append the data to the list
                date = 'Not Available'
                # Try to find the next sibling 'div' which might contain the date
                date_div = div.find_next_sibling('div', class_='ast-col-lg-1')
                if date_div:
                    date_span = date_div.find('span', class_='text-monospace')
                    if date_span:
                        date = date_span.get_text().strip()
                data.append({'URL': url, 'Job Title': job_title, 'Date':date})

    title_url_df = pd.DataFrame(data)
    col_names = ['Company','Designation','Date Listed','Job Type','Job Period','Profession',
             'Industry','Location Name','Allowance / Remuneration','Company Profile',
             'Job Description']
    jobs_info = []
    for url in title_url_df['URL']:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        res_dict = dict.fromkeys(col_names, '')
        for col_name in col_names:
            # Find the div that contains the column name
            col_div = soup.find('div', text=col_name, class_='font-weight-bold')
            if col_div:
                # The actual data is in the next sibling of the parent of col_div
                next_div = col_div.find_next_sibling()
                if next_div:
                    # Extract the text and store it in the dictionary
                    for span in next_div.find_all('span'):
                        span.decompose()
                    res_dict[col_name] = next_div.get_text(strip=True)
        jobs_info.append(res_dict)
    
    # return results 
    internSG_jobs = pd.DataFrame(jobs_info)
    internSG_jobs.to_csv(output_file_path,index=False)
    return internSG_jobs

In [3]:
internSgScrapper(40)

,Company,Designation,Date Listed,Job Type,Job Period,Profession,Industry,Location Name,Allowance / Remuneration,Company Profile,Job Description
0,YouApp Pte Ltd,AI / Ml Research Intern,23 Apr 2024,Entry Level / Junior Executive,"Immediate Start, For At Least 3 Months",IT / Information Technology,Computer and IT,Singapore,"$800 - 1,500 monthly","YouApp integrates Western, Indian, and Eastern...",Our team is made up of people from diverse bac...
1,YouApp Pte Ltd,IT Intern,23 Apr 2024,Entry Level / Junior Executive,"Immediate Start, For At Least 3 Months",IT / Information Technology,Computer and IT,Singapore,"$800 - 1,500 monthly","YouApp integrates Western, Indian, and Eastern...",We are seeking a talented and motivated IT int...
2,YouApp Pte Ltd,AI Intern,23 Apr 2024,Entry Level / Junior Executive,"Immediate Start, For At Least 3 Months",IT / Information Technology,Computer and IT,Singapore,"$800 - 1,500 monthly","YouApp integrates Western, Indian, and Eastern...",Our team is made up of people from diverse bac...
3,Strides Digital,Data Science Intern,22 Apr 2024,Entry Level / Junior Executive,"From Jul 2024, For At Least 6 Months",IT / Information Technology,Computer and IT,"Paya Lebar Road, Paya-lebar Quarter Mall, Sing...","$1,500 monthly",Strides Digital is a company that aims to acce...,Job descriptionWe are looking for data analyti...
4,ClearSK Healthcare Pte Ltd,No Code Developer Intern with AI & Chatbot Exp...,21 Apr 2024,Entry Level / Junior Executive,"Flexible Start, For At Least 6 Months",IT / Information Technology,Healthcare / Fitness / Sports,"Novena, Singapore","$1,400 - 2,200 monthly",Join Our Team and Shape the Future of the Medi...,Job ResponsibilitiesCollaborate with the team ...
...,...,...,...,...,...,...,...,...,...,...,...
766,Up Devlabs Pte. Ltd.,Backend Software Developer Intern,12 May 2023,Entry Level / Junior Executive,"Flexible Start, For At Least 4 Months",IT / Information Technology,Computer and IT,"Fusionopolis Walk, Solaris, Singapore","$800 - 1,000 monthly",UP DevLabs Pte Ltd is currently seeking a moti...,"Responsibilities:Assist in planning, designing..."
767,Up Devlabs Pte. Ltd.,Mobile Application Developer Intern,12 May 2023,Entry Level / Junior Executive,"Flexible Start, For At Least 4 Months",IT / Information Technology,Computer and IT,"Fusionopolis Walk, Solaris, Singapore","$800 - 1,000 monthly",UP DevLabs Pte Ltd is offering an exciting opp...,"Responsibilities:Assist in designing, building..."
768,MIU Global Pte Ltd,Technical Project Leader,12 May 2023,Experienced / Senior Executive,"Immediate Start, Permanent",IT / Information Technology,Creative / Media,Singapore,"$800 - 3,000 monthly",Technical Project Leader & Solution Architect ...,"At MIU Creative Agency, we’re aware of the imp..."
769,Augmentus,Robotics Engineer Intern,12 May 2023,Entry Level / Junior Executive,"Immediate Start, For At Least 6 Months",IT / Information Technology,Computer and IT,"75 Ayer Rajah Crescent, Singapore","$800 - 1,500 monthly",Augmentus is a global robotics software compan...,About your roleDevelop software components for...
